# 364.0{21,22,41-45} Introduction to Image Processing

In [ ]:
import numpy as np

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
import skimage

## Load and Display Images

The scitki-image package comes along with some example images that can be accessed via the `data` module.

In [ ]:
img = skimage.data.cat()  # Example image from the scikit-image package

In [ ]:
plt.figure()
plt.imshow(img)

Moreover, images can be loaded from the drive (or web) using the `imread` function from the `io` module. This module defines an interface for different packages that support io-operations for different image type.

The following function lists all available packages and their supported functions:

In [ ]:
skimage.io.find_available_plugins()

The default package used by the `io` module is the `imageio` package (except for `.tiff`-files the `tifffile` package is used).

Let's try to load some images next and display them using matplotlib.

In [ ]:
img_drive = skimage.io.imread('example.png')    # Read an image from the drive
img_web = skimage.io.imread('https://www.jku.at/fileadmin/_processed_/5/7/csm_erich-kobler_b598995438.jpg')

# display the images
fig, ax = plt.subplots(1,2)
ax[0].imshow(img_drive)
ax[1].imshow(img_web)

In [ ]:
# Let's add titles and turn of the axis clutter.
ax[0].set_title('Logo')
ax[1].set_title('Erich Kobler')
for a in ax: a.axis('off')

Let us check the size of the images. It should be number of rows times number of columns times the number of color channels, i.e. $N\times M \times C$.

In [ ]:
print(img.shape)
print(img_drive.shape)

In [ ]:
# We do not need the 4th channel now, so let's remove it.
img_drive = img_drive[..., :3]

Why has `img_drive` 4 channels?

Typically images are stored in the RGB color space. So, the channel represent the Red, Green, and Blue color component of a pixel. The next figure displays each channel individually.

In [ ]:
fig, ax = plt.subplots(1, img_drive.shape[2], sharex=True, sharey=True)
labels = ['red', 'green', 'blue']
for i, a in enumerate(ax):
    a.imshow(img_drive[..., i], cmap='gray')
    a.set_title(labels[i])
    a.axis('off')

### Image data types

Since images are represented as numpy arrays, their data (pixel intensity values) may be stored using the supported `dtypes`.
Typically images are stores using the 8bit unsigned int data type (i.e. `np.uint8`). However, image processing is often performed using floating point values in a normalized interval such as $[0,1]$.

In [ ]:
# Let's check the dtype of the loaded images.
print("dtype:", img_drive.dtype)
print(f"Range: [{img_drive.min()}, {img_drive.max()}]")

Convert it to floating point variables in $[0,1]$ using single precision to avoid unwanted quantization artifacts.

In [ ]:
img_drive = img_drive.astype(np.float32)/255
print("dtype:", img_drive.dtype)
print(f"Range: [{img_drive.min()}, {img_drive.max()}]")

Since the image is represented as a numpy array, its entries (pixel intensities) can be manipulated using numpy functions. Note also the function `img_as_float` can be used to convert the dtype from `np.uint8` to `np.float`.

In [ ]:
img_sine = np.sin(2*np.pi*skimage.img_as_float(img)) # 
fig, ax = plt.subplots(1,2)
ax[0].imshow(img)
ax[1].imshow(img_sine)

Finally, the `io` module allows us to export (save) images using the function `imsave`.

In [ ]:
print('Without a conversion we get the following warning:', flush=True)
skimage.io.imsave('cat-sine.jpg', img_sine, quality=90) # The quality argument is optional for the JPEG format.

print('With a conversion no warning is displayed.', flush=True)
skimage.io.imsave('cat-sine.jpg', ((img_sine + 1)/2*255).astype(np.uint8), quality=90) # The quality argument is optional for the JPEG format.

For more details on the `io` module have a look at the [documentation](https://scikit-image.org/docs/dev/api/skimage.io.html#skimage.io).

## Color conversion

Many image processing algorithms require that images are represented in certain color spaces such as RGB, HSV, Lab, or YCbCr. Scikit-image provides the `color` module to convert pixel data between these spaces. 
The naming convention of the conversion functions is rather simple: 

>    `<orign>2<target>`, 

where `<origin>` is the name of the color space to convert from and `<target>` is the name of the destination color space. You find some examples below.

In [ ]:
color_conversion = {'RGB': lambda x: x,
                    'Gray': skimage.color.rgb2gray,
                    'HSV': skimage.color.rgb2hsv,
                    'Lab': skimage.color.rgb2lab,
                   }
img_cc = {k: v(img_drive) for k, v in color_conversion.items()}

Next, we check the range of the different color spaces for the particular image.

In [ ]:
# A pretty printing function for arrays
def pp_array(x):
    return np.array2string(x, formatter={'float': lambda x: f'{x:.2f}'})


for k, v in img_cc.items():
    print(f"{k:4s} in [{pp_array(np.min(v, (0,1))):20s}, {pp_array(np.max(v, (0,1))):20s}]")


Due to the larger range of the `Lab` color space, we will get a warning when displaying images using matplotlib.

In [ ]:
fig, ax = plt.subplots(2, len(img_cc)//2, sharex=True, sharey=True)
ax = ax.ravel()

for i, (k, v) in enumerate(img_cc.items()):
    ax[i].imshow(v, cmap='gray')   # The cmap attribute is just used for the 'gray' image.
    ax[i].set_title(k)

However, we can avoid this warning by normalizing the channels.

In [ ]:
def normalize_channels(x):
    x = x.copy()
    x -= x.min((1,2), keepdims=True)
    x /= x.max((1,2), keepdims=True)
    return x

fig, ax = plt.subplots()
ax.imshow(normalize_channels(img_cc['Lab']))

Further details on color channels can be found at [wikipedia](https://en.wikipedia.org/wiki/Color_space) and the API of the `color` module is described in the [documentation](https://scikit-image.org/docs/dev/api/skimage.color.html#skimage.color).

## Interpretation of Images

Up to now, we interpreted images as 2-dimensional matrices with scalar (gray-scale) or vector valued entries (color). However, images can also be interpreted as discretizations of continuous objects. Thus, images may represent `continuous' functions in two variables $(x,y)\in\Omega\subset\mathbb{R}^2$.

In the next few lines of codes, we elaborate on this idea.

In [ ]:
# load a gray-scale image
x = skimage.data.coins()[10:80, 300:370]

# plot it as usual
fig = plt.figure()
ax = fig.add_subplot(1,2,1)
ax.imshow(x, cmap='gray')
ax.set_title("Matrix interpretation")

# define the center points of the pixels on the [0,1]^2 square.
xx, yy = np.meshgrid(np.linspace(0,1,x.shape[1]), np.linspace(0,1,x.shape[0]))
zz = x[::-1,:]
# plot the function in two variables
ax = fig.add_subplot(1,2,2, projection="3d")
ax.plot_surface(xx, yy, zz, cmap='gray', rcount=x.shape[0], ccount=x.shape[1])
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("Function interpretation")
ax.azim = -90
ax.elev = 90


## Distribution of Intensity Values (Histograms)

The global (across the entire image) discrete distribution of intensity values is called the histogram of an image. Scikit-image provides the `exposure` module to compute and manipulate histograms. Typically, the histogram is not normalized and hence it sums up to the number of pixels of an image.

The cumulative density function (cdf) of an image is the sum of pixels having an intensity value smaller or equal to a defined value weighted by the number of pixels. Thus, the cdf is in the range $[0,1]$.

In [ ]:
img_hist, bins = skimage.exposure.histogram(img[..., 0], nbins=256)  # Compute the histogram of the red color channel
# Note the histogram can be normalized by setting `normalize=True` argument.

img_cdf, bins  = skimage.exposure.cumulative_distribution(img[..., 0], nbins=256)   # Compute the cumulative distribution

fig, ax = plt.subplots(1,2)
ax[0].imshow(img[..., 0], cmap='gray')   # Plot the re channel
ax[1].plot(bins, img_hist)               # Plot the histogram
ax_cdf = ax[1].twinx()                   # Create a twin axis since the cdf is in the range [0,1]
ax_cdf.plot(bins, img_cdf, '-r')         # Plot the cdf.

The cdf of an image is often used to manipulate the histogram of an image. For example, the next piece of code adapts the contrast of an image by equalizing its histogram. 

Further histogram manipulation functions can be found in the [documentation](https://scikit-image.org/docs/0.18.x/api/skimage.exposure.html?highlight=exposure#skimage.exposure) of the `exposure` module.

In [ ]:
img_eq = skimage.exposure.equalize_hist(img[..., 0])

img_hist, bins = skimage.exposure.histogram(img_eq, nbins=256) 
img_cdf, bins  = skimage.exposure.cumulative_distribution(img_eq, nbins=256)   # Compute the cumulative distribution

fig, ax = plt.subplots(1,2)
ax[0].imshow(img_eq, cmap='gray')
ax[1].plot(bins, img_hist)
ax_cdf = ax[1].twinx()
ax_cdf.plot(bins, img_cdf, '-r')

## Geometrical Transformation of Images

Geometrical operations are typically applied to change the shape of an image. Scikit-image summarizes these operations in the `transform` module. We demonstrate the most popular geometrical transformations in the next cells. For additional transforms check out the [documentation](https://scikit-image.org/docs/dev/api/skimage.transform.html).

In [ ]:
import skimage.transform    # the module needs to imported from tha package

Let us start with cropping and resizing operations. Cropping defines the extraction of a patch from an image, while the resizing operation changes the resolution of an image. Lock at the following code and figure to understand the differences.

In [ ]:
img_g = skimage.color.rgb2gray(img)

img_crop = img_g[100:250,100:250]    # Cropping a patch from the image can be done using numpy slicing
img_rescaled = skimage.transform.rescale(img_g, 0.25, anti_aliasing=True)    # Rescale the image to a quater resolution
img_resized = skimage.transform.resize(img_g, (img_g.shape[0]//2, img_g.shape[1]//2))    # Resize the image to half resolution

fig, ax = plt.subplots(2,2)
for a, i, t in zip(ax.ravel(), [img_g, img_crop, img_rescaled, img_resized], ['original', 'cropped', 'rescaled', 'resized']):
    a.imshow(i, cmap='gray')
    a.set_title(t)

In [ ]:
img_r = skimage.transform.rotate(img_g, 45)    # Rotate the image by 45 degrees

plt.figure()
plt.imshow(img_r, cmap='gray')
plt.title('Rotated')

## Boundary Handling

Next, we discuss typical boundary handling approaches used in image processing to suppress boundary artifacts.

In the next table, you find a look table that translates the Numpy border handling modes to the corresponding OpenCV border handling enum items.

| numpy, scikit-image | opencv                  |
|:-------------------:|:-----------------------:|
| `constant`          | `cv.BORDER_CONSTANT`    |
| `edge`              | `cv.BORDER_REPLICATE`   |
| `wrap`              | `cv.BORDER_WRAP`        |
| `reflect`           | `cv.BORDER_REFLECT_101` |
| `symmetric`         | `cv.BORDER_REFLECT`     |


In [ ]:
# create a simple image
x = np.zeros((16, 16))
x[:8, :8] += 1
x[:4, :4] += 1
x[:2, :2] += 1
x[:1, :1] += 2
x[8, 8] = 4

modes = ['constant', 'edge', 'wrap', 'reflect', 'symmetric']
fig, axes = plt.subplots(1, len(modes), figsize=(2*len(modes),3))
ax = axes.flatten()

for n, mode in enumerate(modes):
    x_padded = np.pad(x, pad_width=x.shape[0], mode=mode)
    ax[n].imshow(x_padded, cmap=plt.cm.gray)
    ax[n].plot([15.5, 15.5, 31.5, 31.5, 15.5],
               [15.5, 31.5, 31.5, 15.5, 15.5], 'y--', linewidth=0.5)
    ax[n].set_title(mode)

for a in ax:
    a.set_axis_off()
    a.set_aspect('equal')

## Corrupting Images by Noise

Images are quite frequently (actually almost always) degraded by noise. The `random_noise` function in the `util` module allows us to synthetically degrade images.

In [ ]:
fig, ax = plt.subplots(1,2)
ax[0].imshow(skimage.util.random_noise(img, mode='gaussian', var=0.1**2))    # add Gaussian noise with standard deviation 0.1
ax[1].imshow(skimage.util.random_noise(img, mode='s&p', amount=0.2))         # add Salt and Pepper (Bernoulli) noise with probability 0.2 to every pixel

## Convolution and Image Filtering

To convolve multidimensional signals (images, volumes) `scipy` provides the `convolve` function in its `ndimage` module. Note that you have multiple options, for instance, also the functions `scipy.signal.convolve2d` performs a 2d convolution.



In [ ]:
from scipy.ndimage import convolve
# generate a 2d Gaussian kernel
r = 4        # radius of filter (final size will be 9x9)
sigma = 2.0  # standard deviation of the Gaussian kernel
xx, yy = np.meshgrid(np.arange(-r,r+1), np.arange(-r,r+1))   
gaussian = np.exp(-(xx**2 + yy**2)/2/sigma**2)
gaussian /= gaussian.sum()    # normalize to preserve intensities
img_gaussian = convolve(img_g, gaussian)   # apply the Gaussian kernel

fig, ax = plt.subplots(1, 2, sharex=True, sharey=True)
ax[0].imshow(img_g, cmap='gray', vmin=0, vmax=1)
ax[1].imshow(img_gaussian, cmap='gray', vmin=0, vmax=1)


We can obtain the same result by using the `filters` module of scikit-image. This module implements a large variety of classical image filters. Check out the [documentation](https://scikit-image.org/docs/dev/api/skimage.filters.html) for further details.
The next cells demonstrates how to extract image edges using edge filters.

In [ ]:
import skimage.filters

edge_filters = {
    "Laplace": skimage.filters.laplace(img_g),
    "Sobel": skimage.filters.sobel(img_g),
    "Canny": skimage.feature.canny(img_g)
}

fig, ax = plt.subplots(1, len(edge_filters), sharex=True, sharey=True)
ax = ax.ravel()

for a, (k,v) in zip(ax, edge_filters.items()):
    a.imshow(v, cmap='gray')
    a.set_title(k)

## Fourier Transformation on Images

Many filtering techniques can be efficiently implemented in Fourier space due to the famous circular convolution theorem. Thus, the 2d Fourier transform is quite frequently used in image processing. Numpy provides the `fft` module to compute multidimensional Fourier transforms.

In [ ]:
Img_g = np.fft.fft2(img_g)
Img_g = np.fft.fftshift(Img_g)

fig, ax = plt.subplots(2, 2, sharex=True, sharey=True)
ax[0,0].imshow(np.log(np.abs(Img_g.real))*np.sign(Img_g.real))
ax[0,1].imshow(np.log(np.abs(Img_g.imag))*np.sign(Img_g.imag))
ax[1,0].imshow(np.log(np.abs(Img_g)))
ax[1,1].imshow(np.angle(Img_g))


## Bonus Task (4%)

Implement the abstraction method of Winemöller et al. [1] to apply a comic style to real-world images.
Note that your solution is only supposed to use functions available in the `numpy`, `skimage`, and `cv2` packages. No worries, your solution does not have to be real-time capable.

[1] Winnemöller, Holger, Sven C. Olsen, and Bruce Gooch. ["Real-time video abstraction."](https://cs.colby.edu/courses/S16/cs365/papers/winnemoller-videoAbstraction-SIG06.pdf) ACM Transactions On Graphics (TOG) 25.3 (2006): 1221-1226.



In [ ]:
img = skimage.io.imread('./jku0.png')
plt.figure()
plt.imshow(img)
img = img[..., :3].astype(np.float32)/255

In [ ]:
from abstractify import abstractify

ia = abstractify(img)
plt.figure()
plt.imshow(ia)